# Simplified Multi-Domain Assessment

Intention of this section is to assess the usefulness of *non prioritised* weighting in multi domain assessment. Effectively applying MTMF with individual / subsets of metrics across domains. 

## 1-Metric Matrix

$$
H = [H_0, H_1, \cdots, H_M]^T = 
\left[ 
\begin{array}{cccc}
1 & 0 & \cdots  & 0 \\
0 & 1 & \cdots  & 0 \\
0 & 0 & \cdots  & 1
\end{array}
\right]$$

$$ \begin{align}
  \Delta T_{ix} &= \frac{\sum_{j\neq x}\left( \overline{T_{i,j}}^{\forall t}\right)}{N-1} \\
  \Delta T_{ix}^- &= \frac{\sum_{j\neq x} \Delta T_{ij}}{N-1}
\end{align}
$$

The Performance ($P$) and False Positive Performance ($P^-$) of the system can then be described as

$$ H . 
\left[
\begin{array}{c}
\Delta T_{ix}^0 \\
\Delta T_{ix}^1 \\
\cdots \\
\Delta T_{ix}^M
\end{array}
\right] = P$$

Results generated using Combined Comparisons Run with `max_emphasis` of 2 and `max_sum` of 1

In [6]:
import os
results_path = "/home/bolster/src/aietes/results/Malicious Behaviour Trust Control-2016-02-24-14-40-20"
outlier_path = os.path.join(results_path, "outliers.h5")

In [11]:
import pandas as pd
with pd.getstore() as store:
    print store.keys()

ImportError: No module named pandas

In [13]:
import sys
sys.path

['',
 '/home/bolster/.local/lib/python2.7/site-packages/zodbpickle-0.6.0-py2.7-linux-x86_64.egg',
 '/home/bolster/src/pyweeknote-generator',
 '/home/bolster/.local/lib/python2.7/site-packages/slacker-0.8.6-py2.7.egg',
 '/usr/lib/python2.7/dist-packages',
 '/usr/lib/python2.7',
 '/usr/lib/python2.7/plat-x86_64-linux-gnu',
 '/usr/lib/python2.7/lib-tk',
 '/usr/lib/python2.7/lib-old',
 '/usr/lib/python2.7/lib-dynload',
 '/home/bolster/.local/lib/python2.7/site-packages',
 '/usr/local/lib/python2.7/dist-packages',
 '/usr/lib/python2.7/dist-packages/PILcompat',
 '/usr/lib/python2.7/dist-packages/gtk-2.0',
 '/usr/lib/pymodules/python2.7',
 '/usr/lib/python2.7/dist-packages/wx-3.0-gtk2',
 '/usr/local/lib/python2.7/dist-packages/IPython/extensions',
 '/home/bolster/.ipython']